In [1]:
import os
from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession
import warnings

warnings.filterwarnings("ignore")

# Encontrar o caminho absoluto do diretório atual
notebook_cwd = os.getcwd()

# Definir o caminho correto para a raiz do projeto Kedro
project_path = r"c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project"

# Verificar o diretório atual e o caminho do projeto
print(f"Notebook current working directory: {notebook_cwd}")
print(f"Project path: {project_path}")

# Alterar para o diretório raiz do projeto Kedro
os.chdir(project_path)

# Bootstrap o projeto Kedro
bootstrap_project(project_path)

# Inicialize o contexto do Kedro
with KedroSession.create() as session:
    context = session.load_context()

# Recarregar o catálogo
catalog = context.catalog

# Acessar os parâmetros
params = context.params

# Listar o catálogo
catalog.list()

[09/14/24 19:13:52] INFO     Using                                                                  ]8;id=148454;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=419052;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py#249\249]8;;\
                             'c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framew                
                             ork\project\rich_logging.yml' as logging configuration.                               

Notebook current working directory: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\notebooks\gmferratti\pipeline
Project path: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project


[09/14/24 19:13:55] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=872320;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=863467;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         


[
    'decks_json_partitioned',
    'decks_txt_partitioned',
    'sampled_decks',
    'players',
    'players_with_decks',
    'parameters',
    'params:global',
    'params:global.run_date',
    'params:global.run_date.day',
    'params:global.run_date.month',
    'params:global.run_date.year',
    'params:global.user',
    'params:global.user.project_path',
    'params:preprocessing',
    'params:preprocessing.webscraper',
    'params:preprocessing.webscraper.zip_url',
    'params:preprocessing.webscraper.zip_folder',
    'params:preprocessing.webscraper.log_folder',
    'params:preprocessing.webscraper.deck_cards',
    'params:preprocessing.webscraper.sample_size_ratio',
    'params:simulation',
    'params:simulation.n_players',
    'params:simulation.log_folder',
    'params:simulation.plays_per_player',
    'params:modeling',
    'params:modeling.xgboost',
    'params:modeling.xgboost.learning_rate',
    'params:modeling.xgboost.max_depth',
    'params:modeling.xgboost.n_estimat

In [2]:
from typing import List, Dict
from classes.deck import Deck
from classes.player import Player
from src.mtg_project.pipelines.utils import setup_logger
import pandas as pd
from faker import Faker

In [3]:
def create_players(n_players: int):
    """
    Cria uma lista de objetos Player com nomes aleatórios.

    Args:
        n_players (int): Número de jogadores a serem criados.

    Returns:
        List[Player]: Lista de objetos Player com nomes gerados aleatoriamente.
    """
    # Inicializando o gerador de dados falsos Faker
    fake = Faker()
    
    # Gerando uma lista de nomes aleatórios usando o Faker
    player_names = [fake.first_name() + " " + fake.last_name() for _ in range(n_players)]
    
    # Criando uma lista de objetos Player a partir dos nomes gerados
    players = [Player(name) for name in player_names]

    # Retornando a lista de objetos Player
    return players

n_players = catalog.load("params:simulation.n_players")
players = create_players(n_players)
catalog.save("players", players)

[09/14/24 19:13:56] INFO     Loading data from params:simulation.n_players (MemoryDataset)...   ]8;id=839938;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=475040;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Saving data to players (PickleDataset)...                          ]8;id=533151;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=685786;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

In [4]:
import random
import os
from typing import List, Dict

def assign_decks_to_players(
        players: List[Player], 
        sampled_decks: Dict[str, str],
        log_folder: str) -> List[Player]:
    """
    Função para atribuir decks aleatórios a cada player na lista de players.

    A função tentará atribuir um deck a cada player chamando o método assign_deck().
    Caso ocorra algum erro na atribuição, tentará com outro deck disponível.

    Args:
        players (list): Lista de objetos Player.
        sampled_decks (dict): Dicionário com os nomes e caminhos dos decks.
        log_folder (str): Caminho da pasta para salvar o log.

    Returns:
        List[Player]: Lista de objetos Player com decks atribuídos.
    """
    # Caminho do arquivo de log
    log_filepath = os.path.join(log_folder, 'decks_assignment.txt')

    # Cria a pasta de log se ela não existir
    os.makedirs(log_folder, exist_ok=True)

    # Configura o logger geral
    logger = setup_logger("validate_decks", log_filepath)
    
    # Log de início da validação
    logger.info("Validating decks...")

    # Convertemos as chaves do dicionário para uma lista de nomes de decks disponíveis
    available_decks = list(sampled_decks.keys())
    
    for player in players:
        assigned = False
        while not assigned and available_decks:
            try:
                # Seleciona um deck aleatório da lista de decks disponíveis
                deck_name = random.choice(available_decks)

                # Obter o caminho completo do deck a partir do dicionário sampled_decks
                deck_path = sampled_decks[deck_name]

                # Cria um novo objeto Deck
                deck = Deck()

                # Carrega o deck a partir do arquivo .txt no caminho obtido
                deck.load_deck_from_txt(deck_path)

                # Atribui o deck ao player
                player.assign_deck(deck)
                logger.info(f"Deck '{deck_name}' assigned to player '{player.name}'")
                
                # Remove o deck da lista de decks disponíveis para evitar reutilização
                available_decks.remove(deck_name)

                assigned = True  # Deck atribuído com sucesso
            except Exception as e:
                # Em caso de erro, tenta outro deck
                logger.error(f"Failed to assign deck '{deck_name}' to player '{player.name}': {e}")
                continue
        
        
        # Se não houver mais decks disponíveis e não conseguir atribuir, lança um erro
        if not assigned:
            raise ValueError(f"No available decks left to assign to player '{player.name}'.")

    logger.info("Deck assignment process completed.")

    return players

players = catalog.load("players")
sampled_decks = catalog.load("sampled_decks")
log_folder = catalog.load("params:simulation.log_folder")
players_with_decks = assign_decks_to_players(players, sampled_decks, log_folder)
catalog.save("players", players_with_decks)

                    INFO     Loading data from players (PickleDataset)...                       ]8;id=659808;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=689170;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from sampled_decks (PickleDataset)...                 ]8;id=598251;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=667086;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from params:simulation.log_folder (MemoryDataset)...  ]8;id=966202;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=309851;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

[09/14/24 19:14:50] INFO     Saving data to players (PickleDataset)...                          ]8;id=227027;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=751593;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

In [16]:
def simulate_plays():
    return 0

plays_per_player = catalog.load("params:simulation.plays_per_player")    

Deck(60 cards, 25 lands, Language: EN, Sets: All sets allowed, Colors: All colors allowed)